# Αριθμητική Ανάλυση
## Εργασία 3: Παρεμβολή, παλινδρόμηση, εύρεση ριζών

### Σημαντικές σημειώσεις

- Όι ζητούμενες συναρτήσεις δεν πρέπει έχουν αναφορές σε global μεταβλητές εκτός από σταθερές καθιερωμένων βιβλιοθηκών (π.χ numpy.pi).
- Ο αριθμός μητρώου πρέπει να είναι σύμφωνος με αυτόν που έχει δηλωθεί στο προφίλ του eclass.
- Όπου χρησιμοποιούνται ψηφία του αριθμού μητρώου, πρέπει να είναι απευθείας δηλωμένα στον κώδικα (hard-coded) και όχι να εξάγονται από τον ΑΜ.
- Το αρχείο της εργασίας πρέπει να τρέχει εξ ολοκλήρου (Run All) για να βαθμολογηθεί.


In [3]:
# Εισαγωγή μονάδων κώδικα
# - Μην αλλάζετε αυτό το κελί.

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d
from scipy.optimize import bisect, newton


In [4]:
# Στοιχεία φοιτητή
# - Εισάγετε το όνομά σας με ελληνικούς χαρακτήρες.

onoma = "ΘΕΟΔΩΡΟΣ-ΚΟΣΜΑΣ"
eponymo = "ΓΟΥΝΕΛΑΣ"
AM = 3121207


### Άσκηση 1

Από την [δυναμομέτρηση ενός κινητήρα εσωτερικής καύσης](https://www.sciencedirect.com/science/article/pii/S0263876213000853) προέκυψε ο παρακάτω πίνακας Ταχύτητας Περιστροφής ($Ν$)- Μέγιστης Ροπής ($τ$):

|Ταχύτητα Περιστροφής [rpm]|Μέγιστη Ροπής  [Nm]|
|---|---|
|1000|$160.8+2a$|
|1500|$249.4+2b$|
|2000|$326.6+2c$|
|2500|$313.3$|
|3000|$198.8$|
|3500|$199.6$|

όπου a,b,c τα 3 τελευταία ψηφία του αριθμού μητρώου σας (c=τελευταίο). Η καμπύλη που ορίζεται από αυτά τα σημεία ονομάζεται και περιβάλλουσα.

Γράψτε μια συνάρτηση με το όνομα `myfunction1` η οποία δέχεται ως είσοδο μία τιμή ταχύτητας περιστροφής $Ν_{in}$ και υπολογίζει στο σημείο αυτό την μέγιστη ροή του κινητήρα με

1) γραμμική παρεμβολή ,
2) παρεμβολή φυσικών κυβικών spline, 
3) παρεμβολή πολυώνυμου 5ου βαθμού και
4) παλινδρόμηση πολυώνυμου 3ου βαθμού. 

Η συνάρτηση πρέπει να επιστρέφει ένα διάνυσμα $τ_{out}$ με τις αυτές τις 4 τιμές. Σε όλες τις περιπτώσεις πρέπει να επιστρέφει μηδενικές τιμές εκτός του πεδίου λειτουργίας του κινητήρα. 
 
Ελέγξτε τα αποτελέσματα της συνάρτησης για τις τιμές ταχύτητας περιστροφής:
- 700 rpm
- 1200 rpm
- 2300 rpm

In [5]:
a = int(str(AM)[-3])
b = int(str(AM)[-2])
c = int(str(AM)[-1])

# Πίνακας με τα σημεία 
points = [(1000, 160.8+2*a), (1500, 249.4+2*b), (2000, 326.6+2*c), (2500, 313.3), (3000, 198.8), (3500, 199.6)]

# Έλεγχος αν η τιμή είναι εντός του εύρους λειτουγίας του κινητήρα
def in_range(Nin):
    if Nin < points[0][0] or Nin > points[-1][0]:
        return False
    return True

# Γραμική Παρεμβολή
def linear_interpolation(Nin):
    # Βρίσκουμε τα δύο σημέια στα οποία βρίσκεται η τιμή 
    # και υπολογίζουμε το αποτέλεσμα σύμφωνα με το τύπο y = (x - x₁) * (y₂ - y₁) / (x₂ - x₁) + y₁
    for i in range(len(points)-1):
        if Nin >= points[i][0] and Nin <= points[i+1][0]:
            x1, y1 = points[i]
            x2, y2 = points[i+1]
            y = (Nin - x1) * (y2 - y1) / (x2 - x1) + y1
            return y      
    return -1

# Κυβική Spline
def cubic_spline(Nin):
        x = [point[0] for point in points]
        y = [point[1] for point in points]
        # Δημιουργία αντικειμένου CubicSpline
        cs = CubicSpline(x, y, bc_type='natural')

        for i in range(len(points)-1):
            if Nin >= points[i][0] and Nin <= points[i+1][0]:
                return cs(Nin)    
            
        return -1

# Παρεμβολή 5ου βαθμού
def linear_interpolation_5th(Nin):
    x = [point[0] for point in points]
    y = [point[1] for point in points]
    # Υπολογισμός την παρεμβολής πολυωνύμου 5ου βαθμού
    poly_coeffs = np.polyfit(x, y, 5)
    
    # Υπολογισμός της τιμής του πολυωνύμου στο σημείο Nin
    poly_value = np.polyval(poly_coeffs, Nin)
    
    return poly_value

# Παρεμβολή 3ου βαθμού με παλινδρόμηση
def linear_interpolation_with_regression(Nin):
    x = [point[0] for point in points]
    y = [point[1] for point in points]
    
    # Υπολογισμός των συντελεστών του πολυωνύμου 3ου βαθμού
    poly_coeffs = np.polyfit(x, y, 3)
    
    # Υπολογισμός της τιμής του πολυωνύμου στο σημείο Nin
    # Η παλινδρόμηση του πολυωνύμου γίνεται αυτόματα απο την np.polyval
    poly_value = np.polyval(poly_coeffs, Nin)
    
    return poly_value 

def myfunction1(Nin):
    result = []
    if not in_range(Nin):
        result = [0, 0, 0, 0]
    else:
        result.append(linear_interpolation(Nin))
        result.append(float(cubic_spline(Nin)))
        result.append(linear_interpolation_5th(Nin))
        result.append(linear_interpolation_with_regression(Nin))
    return result

In [6]:
# Πρόχειρο άσκησης 1
# - Σε αυτό το κελί μπορείτε να τυπώσετε μεταβλητές και να κάνετε γραφικές παραστάσεις για επαλήθευση. Δεν λαμβάνεται υπόψη στην βαθμολόγηση.
print(f"Nin : 700  -> {myfunction1(500)} ")
print(f"Nin : 1200 -> {myfunction1(1200)}")
print(f"Nin : 2500 -> {myfunction1(2300)}")


Nin : 700  -> [0, 0, 0, 0] 
Nin : 1200 -> [198.64000000000001, 195.9343157894737, 183.7786496000008, 219.54008888888916]
Nin : 2500 -> [324.22, 342.08357894736844, 340.1125503999996, 305.8828952380952]


### Άσκηση 2

Η θερμοκρασία $Τ$ της επιφάνειας ενός θερμαντικού σώματος μπορεί να υπολογιστεί από το ισοζύγιο ενέργειας: 

$$
\dot{q}=h_c A (T-T_a)+σ ε Α (Τ^4-{Τ_s}^4)
$$
όπου οι θερμοκρασίες $Τ$ σε Kelvin.

Γράψτε μια συνάρτηση `myfunction2` που δέχεται ως ορίσματα τα βαθμωτά μεγέθη $\dot{q},h_c,A,Τ_a,σ,ε,Τ_s$ και επιστρέφει ένα διάνυσμα με τρεις τιμές θερμοκρασίας $Τ$ που αντιστοιχούν στην επίλυση της εξίσωσης με την μέθοδο:

1) διχοτόμησης με όρια $300$ και $1000\;Κ$.
2) Newton-Raphson με αρχική τιμή $300\;Κ$.
3) τέμνουσας με αρχικές τιμές $300$ και $310\;Κ$.

Τα αποτελέσματα να υπολογιστούν με απόλυτη ακρίβεια $10^{-3}$ και να ελεγχθούν για τις σταθερές τιμές:
- $h_c=5+0.5a\; \frac{W}{m^2K}$, συντελεστής συναγωγής,
- $A=2-0.1b\; m^2 $, επιφάνεια του θερμαντικού σώματος,
- $Τ_a=273+18-0.5c\; K$, θερμοκρασία αέρα του περιβάλλοντος χώρου,
- $σ=5.67\cdot 10^{-8} \;\frac{W}{m^2K^4}$, σταθερά Stefan-Boltzmann,
- $ε=0.8$, ο συντελεστής εκπομπής,
- $Τ_s =10\; °C=10+273=283\; K$, θερμοκρασία των επιφανειών του περιβάλλοντος χώρου

και για την μεταβλητή τιμή:
- $\dot{q}= 500, 750, 1000 \; W$, ηλεκτρική ισχύς.

Η είσοδος και έξοδος τιμών να γίνεται στις μονάδες που δίνονται παραπάνω (θερμοκρασία σε Κ).

In [7]:
# Λύση της άσκησης 2
# - Σε αυτό το κελί γράψτε μόνο τις ζητούμενες συναρτήσεις χωρίς print και input.
a = int(str(AM)[-3])
b = int(str(AM)[-2])
c = int(str(AM)[-1])

def myfunction2(q, hc, A, Ta, sigma, epsilon, Ts):
    # Define the function for which we are trying to find the roots
    def f(T):
        return q - hc * A * (T - Ta) - sigma * epsilon * A * (T**4 - Ts**4)

    # Define the derivative of the function for Newton-Raphson method
    def df(T):
        return -hc * A - 4 * sigma * epsilon * A * T**3

    # Bisection method
    T1 = bisect(f, 300, 1000, xtol=1e-3)

    # Newton-Raphson method
    T2 = newton(f, 300, df, tol=1e-3)

    # Secant method
    T3 = newton(f, x0=300, x1=310, tol=1e-3)

    return [T1, T2, T3]

# Constants
hc = 5 + 0.5 * a # W/m^2K
A = 2 - 0.1 * b  # m^2
Ta = 273 + 18 - 0.5 * c  # K
sigma = 5.67e-8  # W/m^2K^4
epsilon = 0.8
Ts = 10 + 273  # K


In [8]:
# Πρόχειρο άσκησης 2
# - Σε αυτό το κελί μπορείτε να τυπώσετε μεταβλητές και να κάνετε γραφικές παραστάσεις για επαλήθευση. Δεν λαμβάνεται υπόψη στην βαθμολόγηση.

# Variable values
q_values = [500, 750, 1000]  # W

for q in q_values:
    print(myfunction2(q, hc, A, Ta, sigma, epsilon, Ts))

[308.8606834411621, 308.86081712169033, 308.86081709686107]
[319.6072578430176, 319.60754208954154, 319.6075420905272]
[329.84113693237305, 329.84152083179254, 329.8415209128245]
